In [12]:
import torch
from torch import nn
from evotorch import Problem
from evotorch.logging import StdOutLogger
from evotorch.algorithms import SNES
from evotorch.algorithms import CEM
from evotorch.algorithms import GeneticAlgorithm
from evotorch.algorithms import XNES
from evotorch.algorithms import GeneticAlgorithm
from evotorch.decorators import vectorized
from evotorch.operators import TwoPointCrossOver
from evotorch.operators import OnePointCrossOver, GaussianMutation
from evotorch.operators import SimulatedBinaryCrossOver
from evotorch.operators import PolynomialMutation
import numpy as np
import random



In [13]:
# sheath_gas = 30 #range? units?
# aux_gas = 10 #range? units?
# sweep_gas = 0,5 #units?
# spray_voltage = 3,5 #kV
# capillary_temperature = 275 #range? celcius
# capillary_voltage = 10 #range? volts
# tube_lens_voltage = 40 #range? volts

In [14]:
# #prompt for output
def prompt(parameters: torch.Tensor) -> torch.Tensor:
    #prompt user for input from jupyter notebook
    print('Please run instrument with parameters {}'.format(parameters))
    return torch.tensor(float(input("Enter a number: ")))


In [15]:
problem = Problem(
    'max',
    prompt,
   # fitness,
    solution_length = 7,
    #initial_bounds=([35,50],[45,100]),
    bounds=(
    [25,5,0,3,250,8,35],
    [35,15,5,5,290,12,45]
            ),
    dtype=torch.float32,
    #device='cpu'
)
problem
#1 sheath_gas = 30 #range? units?
#2 aux_gas = 10 #range? units?
#3 sweep_gas = 0,5 #units?
#4 spray_voltage = 3,5 #kV
#5 capillary_temperature = 275 #range? celcius
#6 capillary_voltage = 10 #range? volts
#7 tube_lens_voltage = 40 #range? volts

[2023-03-04 07:53:03] INFO     < 4891> evotorch.core: Instance of `Problem` (id:140636753713856) -- The `dtype` for the problem's decision variables is set as torch.float32
[2023-03-04 07:53:03] INFO     < 4891> evotorch.core: Instance of `Problem` (id:140636753713856) -- `eval_dtype` (the dtype of the fitnesses and evaluation data) is set as torch.float32
[2023-03-04 07:53:03] INFO     < 4891> evotorch.core: Instance of `Problem` (id:140636753713856) -- The `device` of the problem is set as cpu
[2023-03-04 07:53:03] INFO     < 4891> evotorch.core: Instance of `Problem` (id:140636753713856) -- The number of actors that will be allocated for parallelized evaluation is 0


In [16]:
def my_own_gaussian_mutation(x: torch.Tensor) -> torch.Tensor:
    # The default GaussianMutation of EvoTorch does not (yet) support different standard deviation values
    # per variable. However, we can define our own mutation operator which adds noise of different magnitudes
    # to different variables, like in this example:
    [_, solution_length] = x.shape
    dtype = x.dtype
    device = x.device

    # Generate Gaussian noise where each column has its own magnitude
    noise = (
        torch.randn(solution_length, dtype=dtype, device=device)
        * torch.tensor([2, 1, 0.5,1,5,1,1], dtype=dtype, device=device)
    )

    return x + noise

In [17]:
searcher = GeneticAlgorithm(
    problem,
    popsize=1,
    operators=[my_own_gaussian_mutation]
       #TwoPointCrossOver(problem, tournament_size=1)
   # ],
)

_ = StdOutLogger(searcher, interval=1)

searcher.run(num_generations=1)


Please run instrument with parameters ReadOnlyTensor([ 27.8095,  12.1159,   3.4912,   4.1697, 288.3546,  11.7927,
                 37.5255])
Please run instrument with parameters ReadOnlyTensor([ 25.6031,  11.1433,   2.6479,   5.3608, 286.6487,  11.0115,
                 37.4077])
         iter : 1
  median_eval : 45.0
    mean_eval : 45.0
pop_best_eval : 45.0
    best_eval : 45.0
   worst_eval : 4.0



In [18]:
list(searcher.population[:10])

[<Solution values=tensor([ 25.6031,  11.1433,   2.6479,   5.3608, 286.6487,  11.0115,  37.4077]), evals=tensor([45.])>]